In [23]:
import requests
import json
import pandas as pd

def get_markets(api_key, secret, passphrase):
    url = "https://clob.polymarket.com/markets"
    headers = {
        'Authorization': f'Bearer {api_key}',
        # Add HMAC signature logic here
    }
    response = requests.get(url, headers=headers)
    return response.json()

In [31]:
# %% [markdown]
# # Fetch all Polymarket markets into a DataFrame

# %% [code]
import requests
import pandas as pd
import time

def fetch_markets(limit=1000, offset=0, closed=None):
    """
    Fetch markets from Polymarket Gamma API.
    Returns list of market dicts.
    """
    base_url = "https://gamma-api.polymarket.com/markets"
    params = {
        "limit": limit,
        "offset": offset
    }
    if closed is not None:
        params["closed"] = str(closed).lower()
    resp = requests.get(base_url, params=params)
    resp.raise_for_status()
    return resp.json()

# %% [code]
# Example: fetch first page
markets_page = fetch_markets(limit=1000, offset=0, closed=False)
len(markets_page)



500

In [35]:

# %% [code]
# Loop through pages to build full list (or until some maximum)
all_markets = []
limit = 1000
offset = 0
while True:
    print(f"Fetching offset {offset}")
    page = fetch_markets(limit=limit, offset=offset, closed=False)
    if not page:
        break
    all_markets.extend(page)
    # if fewer than limit returned, no more pages
    if len(page) < limit:
        break
    offset += limit
    # be polite with API
    time.sleep(0.2)

# %% [code]
# Create DataFrame
df = pd.DataFrame(all_markets)
    
# %% [code]
# Select a subset of useful columns, rename if you like
cols = [
    "id",
    "question",
    "lastTradePrice",
    "slug",
    "conditionId",
    "startDate",
    "endDate",
    "volumeNum",
    "liquidityNum",
    "active",
    "closed",
    "endDate"
]
# Some fields may be missing — use .get or fillna
df_sel = df.reindex(columns=cols)
df_sel = df_sel.rename(columns={
    "volumeNum": "volume",
    "liquidityNum": "liquidity"
})

# %% [code]
# Display sorted by volume descending
df_sel = df_sel.sort_values(by=["volume"], ascending=False)
df_sel.head(20)

# %% [markdown]
# ## Further possibilities
# - You can filter by category (e.g., df_sel[df_sel["category"]=="Politics"]).
# - Convert date columns to pandas datetime: `pd.to_datetime(df_sel["startDate"])`.
# - Export to CSV: `df_sel.to_csv("polymarket_markets.csv", index=False)`.
# - For live updates you could re-run the fetch periodically, or use the WebSocket endpoints.

Fetching offset 0


,id,question,lastTradePrice,slug,conditionId,startDate,endDate,volume,liquidity,active,closed,endDate
303,525362,Will George Russell be the 2025 Drivers Champion?,0.002,will-george-russell-be-the-2025-drivers-champion,0x7e8953cc66f9dabfe80fb0d1a9284bffb7a293f700b21027261fade58a082d73,2025-02-26T19:34:12.998Z,2025-12-07T12:00:00Z,2.613836e+07,520467.43948,True,False,2025-12-07T12:00:00Z
10,516719,Russia x Ukraine ceasefire in 2025?,0.140,russia-x-ukraine-ceasefire-in-2025,0x8ee2f1640386310eb5e7ffa596ba9335f2d324e303d21b0dfea6998874445791,2024-12-29T22:51:33.18Z,2025-12-31T12:00:00Z,2.456885e+07,393511.29970,True,False,2025-12-31T12:00:00Z
189,520930,Jerome Powell out as Fed Chair in 2025?,0.037,will-trump-remove-jerome-powell,0x01d5f475cb30704216ae9906d369c0b2991cafec882ab4888a66ff11eb03e569,2025-01-29T23:13:22.244Z,2025-12-31T12:00:00Z,1.061825e+07,118398.98956,True,False,2025-12-31T12:00:00Z
98,517014,Will The Fantastic Four: First Steps be the top grossing movie of 2025?,0.002,will-the-fantastic-four-first-steps-be-the-top-grossing-movie-of-2025,0x1cfde498b977cf83dd6a0276fb287de98dbeb79554c61368d074736730883ad8,2025-01-02T18:47:24.03753Z,2025-12-31T12:00:00Z,1.038551e+07,428304.24625,True,False,2025-12-31T12:00:00Z
2,516710,US recession in 2025?,0.040,us-recession-in-2025,0xfa48a99317daef1654d5b03e30557c4222f276657275628d9475e141c64b545d,2025-01-08T01:33:54.924Z,2026-02-28T12:00:00Z,9.875202e+06,90725.18390,True,False,2026-02-28T12:00:00Z
102,517018,Will Thunderbolts be the top grossing movie of 2025?,0.004,will-thunderbolts-be-the-top-grossing-movie-of-2025,0x46fb01da8662e5eb2496b600bbe874d230f05f3e99b382f2791015ef55b464af,2025-01-02T18:49:13.002797Z,2025-12-31T12:00:00Z,9.729959e+06,398693.95846,True,False,2025-12-31T12:00:00Z
100,517016,Will Captain America: Brave New World be the top grossing movie of 2025?,0.002,will-captain-america-new-world-order-be-the-top-grossing-movie-of-2025,0x2993e8c18922f93787756e02dc262c193b79f05c7b952a0c9656e948f9977c88,2025-01-02T18:47:42.296Z,2025-12-31T12:00:00Z,9.296330e+06,449629.48083,True,False,2025-12-31T12:00:00Z
103,517019,Will How to Train Your Dragon be the top grossing movie of 2025?,0.002,will-how-to-train-your-dragon-be-the-top-grossing-movie-of-2025,0x22fa7126aad142e3e7cd7dc6ae6669b86e28ff8ae6f07a57e1a05403c8074b5f,2025-01-02T18:50:19.06847Z,2025-12-31T12:00:00Z,8.077479e+06,463754.97465,True,False,2025-12-31T12:00:00Z
207,521918,Will UCR hold the most seats in the Chamber of Deputies following the 2025 Argentina election?,0.001,will-ucr-hold-the-most-seats-in-the-chamber-of-deputies-following-the-2025-argentina-election,0x937d7afc09f1d2a16a4daa1928798725ba97c809bccac841d0907117dbccceca,2025-02-06T00:03:20.627Z,2025-10-26T12:00:00Z,8.073938e+06,29283.36379,True,False,2025-10-26T12:00:00Z
8,516716,Khamenei out as Supreme Leader of Iran in 2025?,0.120,khamenei-out-as-supreme-leader-of-iran-in-2025,0x1b6f76e5b8587ee896c35847e12d11e75290a8c3934c5952e8a9d6e4c6f03cfa,2024-12-29T22:39:48.361Z,2025-12-31T12:00:00Z,7.781774e+06,82010.84220,True,False,2025-12-31T12:00:00Z


In [36]:
# List all columns in the dataframe
print(df.columns.tolist())

# Or for a prettier vertical view
df.info()

# See one example record to inspect nested keys
df.iloc[0]

['id', 'question', 'conditionId', 'slug', 'resolutionSource', 'endDate', 'startDate', 'image', 'icon', 'description', 'outcomes', 'active', 'closed', 'marketMakerAddress', 'createdAt', 'updatedAt', 'new', 'featured', 'submitted_by', 'archived', 'restricted', 'groupItemTitle', 'groupItemThreshold', 'enableOrderBook', 'orderPriceMinTickSize', 'orderMinSize', 'startDateIso', 'volume1wk', 'volume1mo', 'volume1yr', 'umaBond', 'umaReward', 'volume24hrAmm', 'volume1wkAmm', 'volume1moAmm', 'volume1yrAmm', 'volume24hrClob', 'volume1wkClob', 'volume1moClob', 'volume1yrClob', 'volumeAmm', 'volumeClob', 'liquidityAmm', 'liquidityClob', 'negRisk', 'events', 'ready', 'funded', 'cyom', 'competitive', 'pagerDutyNotificationEnabled', 'approved', 'rewardsMinSize', 'rewardsMaxSpread', 'spread', 'oneDayPriceChange', 'oneHourPriceChange', 'oneWeekPriceChange', 'oneMonthPriceChange', 'oneYearPriceChange', 'lastTradePrice', 'bestBid', 'bestAsk', 'clearBookOnStart', 'manualActivation', 'negRiskOther', 'umaRes

id                                                       502517
question                         ARCH Will the match be a draw?
conditionId                                                    
slug                   will-the-match-be-a-draw-romania-ukraine
resolutionSource                                               
                                         ...                   
showGmpOutcome                                              NaN
negRiskMarketID                                             NaN
negRiskRequestID                                            NaN
umaResolutionStatus                                         NaN
deployingTimestamp                                          NaN
Name: 0, Length: 96, dtype: object

In [37]:
df_sports = df_sel[df_sel["category"].str.lower() == "sports"]

# Sort by volume (descending) and show the top ones
df_sports = df_sports.sort_values(by="volume", ascending=False)
df_sports.reset_index(drop=True, inplace=True)

print(f"Found {len(df_sports)} sports markets.")
pd.set_option("display.max_colwidth", 100)
df_sports.head(20)

KeyError: 'category'

In [64]:
# Remove duplicated column names (keep first occurrence)
df_sel = df_sel.loc[:, ~df_sel.columns.duplicated()]

# Make sure endDate is in datetime format
df_sel["endDate"] = pd.to_datetime(df_sel["endDate"], errors="coerce")

# Current UTC time (Polymarket timestamps are UTC)
now = pd.Timestamp.utcnow()

# Compute a 20-minute cutoff
cutoff = now + pd.Timedelta(minutes=20000)

# Filter for markets ending within 20 minutes (but not already ended)
df_ending_soon = df_sel[
    (df_sel["endDate"] > now) & (df_sel["endDate"] <= cutoff)
].copy()

# Sort soonest first
df_ending_soon = df_ending_soon.sort_values(by="endDate")

print(f"{len(df_ending_soon)} markets ending within 20 minutes:")
df_ending_soon[["question", "endDate", "volume", "liquidity"]]

5 markets ending within 20 minutes:


,question,endDate,volume,liquidity
434,Negative GDP growth in Q3 2025?,2025-10-30 12:00:00+00:00,5.223095e+05,50891.78405
436,5% GDP contraction in Q3 2025?,2025-10-30 12:00:00+00:00,3.139119e+05,5229.68446
307,Will the Los Angeles Dodgers win the 2025 World Series?,2025-10-31 12:00:00+00:00,7.566611e+06,187881.50020
306,Will the Toronto Blue Jays win the 2025 World Series?,2025-10-31 12:00:00+00:00,5.198789e+06,194555.42409
457,Will any AI score ≥85% on ARC-AGI-2 by November 3?,2025-11-03 12:00:00+00:00,1.172277e+05,4486.58339
